# Model Summary

In [1]:
from keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.layers import Dense

resnet50 = ResNet50(weights='imagenet')

fc2 = resnet50.get_layer('fc1000').output
prediction = Dense(output_dim=1, activation='sigmoid', name='logit')(fc2)
model = Model(input=resnet50.input, output=prediction)
model.summary()

Using TensorFlow backend.


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 230, 230, 3)   0           input_1[0][0]                    
____________________________________________________________________________________________________
conv1 (Convolution2D)            (None, 112, 112, 64)  9472        zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 112, 112, 64)  256         conv1[0][0]                      
___________________________________________________________________________________________

# Extract feature from one image

In [2]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input
import numpy as np

model = ResNet50(weights='imagenet', include_top=False)

img_path = 'zjg.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

features = model.predict(x)
print (features)

[[[[ 2.18436956  0.54637343  0.31706408 ...,  0.67332363  0.81599325
     0.07851383]]]]


# feature datatype

In [3]:
print (type(features))

<class 'numpy.ndarray'>


In [4]:
print(features.shape)

(1, 1, 1, 2048)


In [5]:
print(features[0][0][0])

[ 2.18436956  0.54637343  0.31706408 ...,  0.67332363  0.81599325
  0.07851383]


In [6]:
print (features[0][0][0].shape)

(2048,)


# Extract Features of all the Image Data

In [7]:
model = ResNet50(weights='imagenet', include_top=False)
out=open("out_id.txt",'r')
img_rep=open("img_rep.txt",'w')
#print (len(out))
features=[]
text=out.readlines()

for id_num in text:
    #id_num=out.readline()
    #COCO_val2014_000000000042
    pre="COCO_val2014_"
    idx=id_num[:-1]
    num0=12-len(idx)
    zero=""
    for i in range(0,num0):
        zero+="0"
    
    idx=pre+zero+idx+".jpg"
    img_path="val2014/"+idx
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    feature = model.predict(x)
    #img_rep.write(str(feature)+'\n')
    features.append(feature[0][0][0])
    #print (features)
    
out.close()
img_rep.close()

In [8]:
# confirm image number
print(len(features))

4857


In [9]:
#transfer it to numpyarray
f=np.array(features)

# K-means Clustering

In [10]:
from sklearn.cluster import KMeans
kmeans=KMeans(n_clusters=12,random_state=0).fit(f)

In [11]:
a=kmeans.labels_

In [12]:
a.shape

(4857,)

In [13]:
outf=open("cluster.txt","w")
for x in a:
    outf.write(str(x)+"\n")
outf.close()

# save the features and clustering labels to a .npy file

In [14]:
np.save('outf',f)

In [15]:
np.save('outa',a)